In [296]:
# !pip3 install cornac

In [297]:
import cornac
from cornac.datasets import amazon_clothing
from cornac.data import ImageModality
from cornac.eval_methods import RatioSplit

In [298]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# we will ignore pandas warning 
import warnings
warnings.filterwarnings('ignore')

In [299]:
ordersExport = pd.read_csv('orders_export_1.csv')
# productsExport = pd.read_csv('products_export.csv')

In [300]:
df_order = ordersExport[['Name', 'Lineitem quantity', 'Lineitem name', 'Lineitem sku']]
print(df_order.shape)
df_order.head()

(3588, 4)


,Name,Lineitem quantity,Lineitem name,Lineitem sku
0,#2962,1,Pink Cotton Shirt - M,HOD0632
1,#2962,1,Dark Blue Cotton Shirt - L,HOD0573
2,#2962,1,Triangular Cotton Shirt - L,HOD0585
3,#2962,1,Red Flower Cotton Shirt - M,HOD0616
4,#2961,1,Suru Kurta - XXL,HOD0282


In [301]:
final= df_order[['Name', 'Lineitem sku', 'Lineitem quantity']]

In [302]:
final['Name'] = final['Name'].str.replace('#', '')

In [303]:
final.head()

,Name,Lineitem sku,Lineitem quantity
0,2962,HOD0632,1
1,2962,HOD0573,1
2,2962,HOD0585,1
3,2962,HOD0616,1
4,2961,HOD0282,1


In [304]:
dele = final[final['Lineitem quantity'] != 1]
print(dele.shape)

(54, 3)


In [305]:
group = final.groupby(['Name', 'Lineitem sku']).sum().reset_index()
print(group.shape)
group.head()

(3472, 3)


,Name,Lineitem sku,Lineitem quantity
0,1002,H0D-K-0001,1
1,1002,H0D-K-0003,1
2,1002,H0D-K-0005,1
3,1002,H0D-K-0010,1
4,1002,HOD-D-0001,1


In [307]:
group['Lineitem quantity'] = group['Lineitem quantity'] / 7

In [308]:
group.tail()

,Name,Lineitem sku,Lineitem quantity
3467,2961,HOD0282,0.142857
3468,2962,HOD0573,0.142857
3469,2962,HOD0585,0.142857
3470,2962,HOD0616,0.142857
3471,2962,HOD0632,0.142857


In [316]:
records = group.to_records(index=False)
result = list(records)
# print(result)

In [310]:
# Define an evaluation method to split feedback into train and test sets
ratio_split = RatioSplit(
    data=records,
    test_size=0.1,
    rating_threshold=0.1,
    exclude_unknowns=True,
    verbose=True,
)

rating_threshold = 0.1
exclude_unknowns = True
---
Training data:
Number of users = 1834
Number of items = 540
Number of ratings = 3124
Max rating = 1.0
Min rating = 0.1
Global mean = 0.2
---
Test data:
Number of users = 179
Number of items = 121
Number of ratings = 205
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 1834
Total items = 540


In [311]:
most_pop = cornac.models.MostPop()

In [312]:
bpr = cornac.models.BPR(
    k=50, max_iter=200, learning_rate=0.001, lambda_reg=0.001, verbose=True
)

In [313]:
wbpr = cornac.models.WBPR(
    k=50, max_iter=200, learning_rate=0.001, lambda_reg=0.001, verbose=True
)

In [314]:
# Use AUC and Recall@20 for evaluation
auc = cornac.metrics.AUC()
rec_20 = cornac.metrics.Recall(k=20)

In [315]:
# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=ratio_split,
    models=[most_pop, bpr, wbpr],
    metrics=[auc, rec_20],
    user_based=True,
).run()


[MostPop] Training started!

[MostPop] Evaluation started!


Ranking:   0%|          | 0/179 [00:00<?, ?it/s]


[BPR] Training started!


  0%|          | 0/200 [00:00<?, ?it/s]

Optimization finished!

[BPR] Evaluation started!


...


[WBPR] Training started!


  0%|          | 0/200 [00:00<?, ?it/s]

Optimization finished!

[WBPR] Evaluation started!


Ranking:   0%|          | 0/179 [00:00<?, ?it/s]


TEST:
...
        |    AUC | Recall@20 | Train (s) | Test (s)
------- + ------ + --------- + --------- + --------
MostPop | 0.8041 |    0.2291 |    0.0009 |   0.0615
BPR     | 0.8227 |    0.2291 |    0.1846 |   0.0677
WBPR    | 0.5698 |    0.0698 |    0.1888 |   0.0687

